# Passive automata learning with LearnLib

This is an example of *passive* automata learning with LearnLib from python using py4j. See https://github.com/LearnLib/learnlib/blob/develop/examples/src/main/java/de/learnlib/examples/passive/Example1.java for the usage of the original Java version.

First, we define an auxiliary to print the learned Mealy machine.

In [1]:
def printDot(hyp, alphabet, gateway):
    # Construct a buffer that we will use to print results on the Python side of our setup
    string_writer = gateway.jvm.java.io.StringWriter()

    # Serialize the hypothesis to the DOT format and write it to the string_writer
    gateway.jvm.net.automatalib.serialization.dot.GraphDOT.write(hyp, alphabet, string_writer,
                                                                  # While varargs allow us to skip this parameter in Java, the method signature expects an array \
                                                                  gateway.new_array(
                                                                      gateway.jvm.net.automatalib.serialization.dot.DOTVisualizationHelper,
                                                                      0))

    print("Learned model in DOT format:")
    print()
    print(string_writer.toString())

Then, we define the gateway to the JVM.

In [2]:
from py4j.java_gateway import JavaGateway, CallbackServerParameters

gateway = JavaGateway(callback_server_parameters=CallbackServerParameters())

# Passive DFA learning

Here, we learn a DFA from training data.

In [3]:
# define the alphabet
alphabet = gateway.jvm.net.automatalib.words.impl.Alphabets.characters('a', 'b')

In [4]:
# instantiate learner
# alternatively one can also use the EDSM variant (BlueFringeEDSMDFA from the learnlib-rpni-edsm artifact)
# or the MDL variant (BlueFringeMDLDFA from the learnlib-rpni-mdl artifact)
learner = gateway.jvm.de.learnlib.algorithms.rpni.BlueFringeRPNIDFA(alphabet)

## Initial automaton without samples

In [5]:
#if no training samples have been provided, only the empty automaton can be constructed
emptyModel = learner.computeModel()
printDot(emptyModel, alphabet, gateway)

Learned model in DOT format:

digraph g {

	s0 [shape="circle" label="0"];

__start0 [label="" shape="none" width="0" height="0"];
__start0 -> s0;
}



## Add positive samples

In [6]:
positiveSamples = gateway.jvm.java.util.ArrayList()
for positiveWord in ["aaa", "aaba", "bba", "bbaba"]:
    positiveSamples.append(gateway.jvm.net.automatalib.words.Word.fromCharSequence(positiveWord))

In [7]:
# add the positive training samples
learner.addPositiveSamples(positiveSamples)
# since RPNI is a greedy state-merging algorithm, providing only positive examples results in the trivial
# one-state acceptor, because there exist no negative "counterexamples" that prevent state merges when
# generalizing the initial prefix tree acceptor
firstModel = learner.computeModel()
printDot(firstModel, alphabet, gateway)

Learned model in DOT format:

digraph g {

	s0 [shape="doublecircle" label="0"];
	s0 -> s0 [label="a"];
	s0 -> s0 [label="b"];

__start0 [label="" shape="none" width="0" height="0"];
__start0 -> s0;
}



## Add negative samples

In [8]:
negativeSamples = gateway.jvm.java.util.ArrayList()
for negativeWord in ["a", "bb", "aab", "aba"]:
    negativeSamples.append(gateway.jvm.net.automatalib.words.Word.fromCharSequence(negativeWord))

In [9]:
# add negative training samples
learner.addNegativeSamples(negativeSamples)

# after adding negative samples (i.e. words that must not be accepted by the model) we get a more "realistic"
# generalization of the given training set
secondModel = learner.computeModel()
printDot(secondModel, alphabet, gateway)

Learned model in DOT format:

digraph g {

	s0 [shape="circle" label="0"];
	s1 [shape="doublecircle" label="1"];
	s2 [shape="circle" label="2"];
	s0 -> s2 [label="a"];
	s0 -> s1 [label="b"];
	s1 -> s1 [label="a"];
	s1 -> s2 [label="b"];
	s2 -> s1 [label="a"];
	s2 -> s0 [label="b"];

__start0 [label="" shape="none" width="0" height="0"];
__start0 -> s0;
}



# Passive Mealy machine learning

Here, we learn Mealy machine from training data.

First, we define the input alphabet.

In [10]:
# define the alphabet
alphabet = gateway.jvm.net.automatalib.words.impl.Alphabets.characters('a', 'b')

In [11]:
# instantiate learner
learner = gateway.jvm.de.learnlib.algorithms.rpni.BlueFringeRPNIMealy(alphabet)

In [12]:
def addSample(learner, inputWord, outputChar):
    learner.addSample(gateway.jvm.net.automatalib.words.Word.fromCharSequence(inputWord), 
                      gateway.jvm.net.automatalib.words.Word.fromCharSequence(outputChar))

We define the training data `trainingData` here. `trainingData` is a list of pairs `(inputWord, outputChar)` such that the output of the SUL for the input word `inputWord` is `outputChar`.

In [13]:
trainingData = [("a", "0"), ("b", "b"), ("aa", "0"), ("ab", "b"), ("ba", "1"), ("bb", "b"), ("baa", "1"), ("bab", "b"), ("bba", "0"), ("bbb", "b"),]

In [14]:
for (inputWord, outputChar) in trainingData:
    addSample(learner, inputWord, outputChar)

We can serialize the learned Mealy machine in DOT format as follows.

In [15]:
mealyModel = learner.computeModel()
printDot(mealyModel, alphabet, gateway)

Learned model in DOT format:

digraph g {

	s0 [shape="circle" label="0"];
	s1 [shape="circle" label="1"];
	s0 -> s0 [label="a / 0"];
	s0 -> s1 [label="b / b"];
	s1 -> s1 [label="a / 1"];
	s1 -> s0 [label="b / b"];

__start0 [label="" shape="none" width="0" height="0"];
__start0 -> s0;
}



The learned Mealy machine is in `CompactMealy` and we can also directly access it.

In [16]:
mealyModel.getClass().getSimpleName()

'CompactMealy'

In [17]:
hyp = mealyModel
# State: http://learnlib.github.io/learnlib/maven-site/0.13.0/apidocs/de/learnlib/algorithms/ttt/base/TTTState.html
# srcID is Integer
for srcID in hyp.getStates():
    for charID in range(hyp.getInputAlphabet().size()):
        transition = hyp.getTransition(hyp.getStateId(srcID), charID)
        dst = hyp.getSuccessor(transition)
        print(f'{srcID} --{alphabet[charID]}/{hyp.getTransitionOutput(transition)}--> {dst}')

0 --a/0--> 0
0 --b/b--> 1
1 --a/1--> 1
1 --b/b--> 0
